In [78]:
import numpy as np
import matplotlib.pyplot as plt
import time
from keras.models import load_model
from tensorflow.keras.datasets import mnist
from tensorflow import keras
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import concatenate, Dense, Flatten, Reshape, Input, Lambda, BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, LeakyReLU, Conv2DTranspose,InputLayer, Reshape

In [79]:
data = np.load('X_false.npy', allow_pickle=True)

In [80]:
BUFFER_SIZE = data.shape[0]
BATCH_SIZE = 5

length = BUFFER_SIZE // BATCH_SIZE * BATCH_SIZE

In [81]:
data = data.astype('float32') / 255.
train_dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [82]:
hidden_dim = 10

def dropout_and_batch(x):
    return Dropout(0.1)(BatchNormalization()(x))

input_img = Input((32, 32, 3))
x = Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu')(input_img)
x = dropout_and_batch(x)
x = Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu')(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = dropout_and_batch(x)

z_mean = Dense(hidden_dim)(x)
z_log_var = Dense(hidden_dim)(x)

In [83]:
def noiser(args):
    global z_mean, z_log_var
    z_mean, z_log_var = args
    N = K.random_normal(shape=(BATCH_SIZE, hidden_dim), mean=0., stddev=1.0)
    return K.exp(z_log_var / 2) * N + z_mean

In [84]:
h = Lambda(noiser, output_shape=(hidden_dim,))([z_mean, z_log_var])

In [85]:
input_dec = Input((hidden_dim))
d = Dense(units=8*8*32, activation='relu')(input_dec)
d = dropout_and_batch(d)
d = Reshape(target_shape=(8, 8, 32))(d)

d = Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')(d)
d = dropout_and_batch(d)
d = Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu')(d)
d = dropout_and_batch(d)
decoded = Conv2DTranspose(filters=3, kernel_size=3, strides=1, padding='same', activation='relu')(d)

encoder = keras.Model(input_img, h, name='encoder')
decoder = keras.Model(input_dec, decoded, name='decoder')
generator = keras.Model(input_img, decoder(encoder(input_img)), name="generator")

In [86]:
discriminator = tf.keras.Sequential()
discriminator.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3]))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.1))

discriminator.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.1))

discriminator.add(Flatten())
discriminator.add(Dense(1))

In [87]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [88]:
def generator_loss(fake_output):
    loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return (loss + kl_loss*0.1)


def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [89]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [77]:
decoder = load_model('decoder')
encoder.load_weights('encoder.h5')
discriminator = load_model('discrim')

In [90]:
@tf.function
def train_step(images):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(images, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [91]:
def draw_img():
    n = 2
    total = 2*n+1

    plt.figure(figsize=(10, 10))

    num = 1
    for i in range(-n, n+1):
          for j in range(-n, n+1):
            ax = plt.subplot(total, total, num)
            num += 1
            img = decoder.predict(np.expand_dims([0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*i/n, 0.5*i/n, 0.5*j/n, 0.5*j/n], axis=0))
            plt.imshow(img.squeeze(), cmap='gray')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

In [92]:
def train(dataset, epochs):
    history = []
    MAX_PRINT_LABEL = 10
    th = BUFFER_SIZE // (BATCH_SIZE*MAX_PRINT_LABEL)
    n_epoch = 1
    for epoch in range(epochs):
        print(f'{n_epoch}/{EPOCHS}: ', end='')

        start = time.time()
        n = 0
        print('**')
        gen_loss_epoch = 0
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)
            gen_loss_epoch += K.mean(gen_loss)
            if( n % th == 0): 
                print('=', end='')
                n += 1

        history += [gen_loss_epoch/n]
        print(': '+str(history[-1]))
        print ('Время эпохи {} в {} секундах'.format(epoch + 1, time.time()-start))
        if (epoch + 1) % 25 == 0:
            discriminator.save('discrim')
            decoder.save('decoder')
            encoder.save('encoder.h5')
        n_epoch += 1
    return history

In [93]:
EPOCHS = 100

In [ ]:
history = train(train_dataset, EPOCHS)

1/100: **
=: tf.Tensor(1906.2515, shape=(), dtype=float32)
Время эпохи 1 в 63.44171690940857 секундах
2/100: **
=: tf.Tensor(1211.7516, shape=(), dtype=float32)
Время эпохи 2 в 79.38197159767151 секундах
3/100: **
=: tf.Tensor(692.2803, shape=(), dtype=float32)
Время эпохи 3 в 76.23341703414917 секундах
4/100: **
=: tf.Tensor(667.57074, shape=(), dtype=float32)
Время эпохи 4 в 79.5133786201477 секундах
5/100: **
=: tf.Tensor(594.8886, shape=(), dtype=float32)
Время эпохи 5 в 69.13490319252014 секундах
6/100: **
=: tf.Tensor(594.30444, shape=(), dtype=float32)
Время эпохи 6 в 75.35840106010437 секундах
7/100: **
=: tf.Tensor(634.6529, shape=(), dtype=float32)
Время эпохи 7 в 50.66230344772339 секундах
8/100: **
=: tf.Tensor(641.5509, shape=(), dtype=float32)
Время эпохи 8 в 55.312886238098145 секундах
9/100: **
=: tf.Tensor(657.0021, shape=(), dtype=float32)
Время эпохи 9 в 43.06359386444092 секундах
10/100: **
=: tf.Tensor(644.8521, shape=(), dtype=float32)
Время эпохи 10 в 45.15541577

In [ ]:
#h = encoder.predict(data, batch_size=BATCH_SIZE)
#plt.scatter(h[:, 0], h[:, 1])

#plt.plot(history)
#plt.grid(True)

# отображение результатов генерации
n = 2
total = 2*n+1

plt.figure(figsize=(10, 10))

num = 1
for i in range(-n, n+1):
      for j in range(-n, n+1):
        ax = plt.subplot(total, total, num)
        num += 1
        img = decoder.predict(np.expand_dims([0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*i/n, 0.5*i/n, 0.5*j/n, 0.5*j/n], axis=0))
        plt.imshow(img.squeeze(), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

In [ ]:
img = decoder.predict(np.expand_dims([0.5*-2/n, 0.5*-2/n, 0.5*1/n, 0.5*-1/n, 0.5*-1/n, 0.5*1/n, 0.5*-1/n, 0.5*-1/n, 0.5*-1/n, 0.5*-2/n], axis=0))
plt.imshow(img.squeeze(), cmap='gray')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

In [ ]:
discriminator.save('discrim')
decoder.save('decoder')
encoder.save('encoder.h5')